<a href="https://colab.research.google.com/github/gustavesg/TCC/blob/main/4_Modelos_Tradicionais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importação drive + bibliotecas

import itertools
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importação base de dados

#base treino normalizada
df_treino = pd.read_csv('/content/drive/MyDrive/TCC/1. Base de Dados/dados_finais_treino/base_final_treino')

#base teste normalizada
df_teste = pd.read_csv('/content/drive/MyDrive/TCC/1. Base de Dados/dados_finais_teste/base_final_teste')

#exclui coluna desnecesária
df_treino = df_treino.drop(columns=['Unnamed: 0'])
df_teste = df_teste.drop(columns=['Unnamed: 0'])



### **Parâmetro Flexibilidade**

In [ ]:
alpha = 0.33

### **Regressão Logística**

In [ ]:
X_treino = df_treino.drop(columns='loan_status')
y_treino = df_treino['loan_status']

# declara lista pra todas variáveis + lista pros subconjuntos de variáveis
todas_variaveis = list(X_treino.columns)
subconjunto_variaveis = []

# BIC inicial definido como infinito
bic_melhor = np.inf
melhor_modelo = None

while todas_variaveis:
    bic_scores = []
    modelos = []

    # Testar cada variável não selecionada
    for var in todas_variaveis:
        subconjunto_teste = subconjunto_variaveis + [var]
        X_possivel_treino = sm.add_constant(X_treino[subconjunto_teste])  # Adiciona a constante para o intercepto

        # Ajustar o modelo de regressão logística
        modelo = sm.Logit(y_treino, X_possivel_treino).fit(disp=0)  # disp=0 para suprimir a saída
        bic_score = modelo.bic  # Obter o BIC do modelo

        bic_scores.append((var, bic_score))
        modelos.append((var, modelo))

    # Selecionar a variável com o menor BIC
    var_melhor, bic_menor = min(bic_scores, key=lambda x: x[1])

    # Se o novo BIC for melhor (menor), adiciona a variável ao modelo
    if bic_menor < bic_melhor:
        bic_melhor = bic_menor
        melhor_modelo = [m[1] for m in modelos if m[0] == var_melhor][0]
        subconjunto_variaveis.append(var_melhor)
        todas_variaveis.remove(var_melhor)
        print(f"Variável '{var_melhor}' adicionada com BIC: {bic_melhor}")
    else:
        break  # Se não houver melhoria no BIC, interrompa o loop

print("Variáveis selecionadas:", subconjunto_variaveis)


Variável 'int_rate' adicionada com BIC: 410025.8020177276
Variável 'dti' adicionada com BIC: 408020.950509354
Variável 'avg_cur_bal' adicionada com BIC: 406424.9302449014
Variável 'loan_amnt' adicionada com BIC: 405499.33825392555
Variável 'total_bc_limit' adicionada com BIC: 404833.2452044925
Variável 'annual_inc' adicionada com BIC: 404637.93427121977
Variáveis selecionadas: ['int_rate', 'dti', 'avg_cur_bal', 'loan_amnt', 'total_bc_limit', 'annual_inc']


In [ ]:
# Trata casos nulos com a mediana
df_treino = df_treino.fillna(df_treino.median())

X_treino = df_treino[list(subconjunto_variaveis)]
y_treino = df_treino['loan_status']

X_teste = df_teste[list(subconjunto_variaveis)]
y_teste = df_teste['loan_status']

# Declara e treina o modelo de regressão logística
model = LogisticRegression()
model.fit(X_treino, y_treino)

# Predições
y_pred = model.predict(X_teste)

# Coeficientes do modelo
coeficientes = model.coef_[0]
print("Coeficientes da Regressão Logística:")
for var, coef in zip(subconjunto_variaveis, coeficientes):
    print(f"{var}: {coef}")

# Teste de Wald
X_treino_const = sm.add_constant(X_treino)  # Adiciona constante para o teste
modelo_stats = sm.Logit(y_treino, X_treino_const).fit()

print("\nTeste de Wald (Significância dos Coeficientes):")
print(modelo_stats.summary())

# Define os pesos para cada classe
pesos = {0: 1 - alpha, 1: alpha}

# Calcula métricas
f1_class_0 = f1_score(y_teste, y_pred, pos_label=0, average='binary')
f1_class_1 = f1_score(y_teste, y_pred, pos_label=1, average='binary')

f1_ponderado = (f1_class_0 * pesos[0] + f1_class_1 * pesos[1]) / sum(pesos.values())
accuracy = accuracy_score(y_teste, y_pred)
precision = precision_score(y_teste, y_pred)
recall = recall_score(y_teste, y_pred)

print("\nF1 Score Ponderado com Pesos Personalizados:", f1_ponderado)
print("Acurácia: ", accuracy)
print("Precisão: ", precision)
print("Sensibilidade: ", recall)





Coeficientes da Regressão Logística:
int_rate: -3.014185762570759
dti: -1.2044948893415448
avg_cur_bal: 0.8611241353114959
loan_amnt: -0.9104940302657374
total_bc_limit: 0.5461714550435359
annual_inc: 0.43124806778098973
Optimization terminated successfully.
         Current function value: 0.636925
         Iterations 5

Teste de Wald (Significância dos Coeficientes):
                           Logit Regression Results                           
Dep. Variable:            loan_status   No. Observations:               317580
Model:                          Logit   Df Residuals:                   317573
Method:                           MLE   Df Model:                            6
Date:                Fri, 22 Nov 2024   Pseudo R-squ.:                 0.08071
Time:                        15:08:34   Log-Likelihood:            -2.0227e+05
converged:                       True   LL-Null:                   -2.2003e+05
Covariance Type:            nonrobust   LLR p-value:                     0.

In [ ]:
#taxa de aprovação
sum(y_pred)/len(y_pred)

0.48155667834227084

### **Random Forest**

In [ ]:
X_treino = df_treino.drop(columns='loan_status')
y_treino = df_treino['loan_status']

X_teste = df_teste.drop(columns='loan_status')
y_teste = df_teste['loan_status']

# Criando e treinando o modelo de Random Forest com pesos nas classes
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_treino, y_treino)

# Fazendo previsões
y_pred = model.predict(X_teste)

#defina os pesos para cada classe
pesos = {0: 1- alpha, 1: alpha}

# Calcule o F1 Score para cada classe
f1_class_0 = f1_score(y_teste, y_pred, pos_label=0, average='binary')
f1_class_1 = f1_score(y_teste, y_pred, pos_label=1, average='binary')

# Calcule o F1 Score ponderado manualmente
f1_ponderado = (f1_class_0 * pesos[0] + f1_class_1 * pesos[1]) / sum(pesos.values())

accuracy = accuracy_score(y_teste, y_pred)
precision = precision_score(y_teste, y_pred)
recall = recall_score(y_teste, y_pred)

print("F1 Score Ponderado com Pesos Personalizados:", f1_ponderado)
print("Acurácia: ", accuracy)
print("Precisão: ", precision)
print("Sensibilidade: ", recall)


In [ ]:
#taxa de aprovação
sum(y_pred)/len(y_pred)

0.33857443972598494